# Quantum states and Lorentz transformations

In this notebook, we will define quantum momentum-polarization states with the `QuantumState` class. We can Lorentz transform these states, and they transform depending on their 4-momenta. The two types of particles we consider here are massive Dirac fermions and photons.

In [1]:
import numpy as np
import qedtool as qtl

## Massive Dirac fermions

The first example is a mixed state of two massive Dirac fermions with opposite 4-momenta $p_\pm = (\varepsilon_\mathbf{p}, \pm\mathbf{p})$. The density matrix of their state is given by $\rho = w_1|\psi_1\rangle\langle\psi_1| + w_2|\psi_2\rangle\langle\psi_2|$ where

$$|\psi_1\rangle = \frac{i}{\sqrt{2}}\Big( |p_+,\mathrm{L};p_-,\mathrm{L}\rangle - |p_+,\mathrm{R};p_-,\mathrm{R}\rangle \Big)\,, \qquad\qquad
|\psi_2\rangle = |p_+,\mathrm{V};p_-,\mathrm{V}\rangle\,,$$

with $w_1 = \frac{4}{5}$ and $w_2 = \frac{1}{5}$. We will first define the 4-momenta, working in keV units:

In [2]:
# Energy and momentum values
m = qtl.constant("electron mass", "keV")
p = 50
ep = np.sqrt(p**2 + m**2)

# 4-momenta
pmu_1 = qtl.FourVector(ep, p, 1.2, 0.3)
pmu_2 = -pmu_1

Next, we define the single particle states $|p_\pm,\mathrm{L}\rangle$, $|p_\pm,\mathrm{R}\rangle$ and $|p_\pm,\mathrm{V}\rangle$:

In [3]:
l_p1 = qtl.QuantumState.single(pmu_1, "L")
l_p2 = qtl.QuantumState.single(pmu_2, "L")
r_p1 = qtl.QuantumState.single(pmu_1, "R")
r_p2 = qtl.QuantumState.single(pmu_2, "R")
v_p1 = qtl.QuantumState.single(pmu_1, "V")
v_p2 = qtl.QuantumState.single(pmu_2, "V")

From the single-particle momentum eigenstates, we form the mixed state:

In [4]:
# Pure components
psi_1 = 1j * (l_p1 * l_p2 - r_p1 * r_p2) / np.sqrt(2)
psi_2 = v_p1 * v_p2
pures = [psi_1, psi_2]

# Classical weights
weights = [4/5, 1/5]

# Mixed state
psi = qtl.QuantumState.mixed(pures, weights)

The mixed state has a density matrix given by

In [5]:
print(psi.rho)

[[ 0.45+0.j -0.05+0.j -0.05+0.j -0.35+0.j]
 [-0.05+0.j  0.05+0.j  0.05+0.j -0.05+0.j]
 [-0.05+0.j  0.05+0.j  0.05+0.j -0.05+0.j]
 [-0.35+0.j -0.05+0.j -0.05+0.j  0.45+0.j]]


We could also compute the $3\times3$ Stokes matrix $S_{ij}$

In [6]:
stokes_matrix = np.array([[qtl.stokes_parameter(psi, [1, 1]), 
                           qtl.stokes_parameter(psi, [1, 2]),
                           qtl.stokes_parameter(psi, [1, 3])],
                          [qtl.stokes_parameter(psi, [2, 1]), 
                           qtl.stokes_parameter(psi, [2, 2]),
                           qtl.stokes_parameter(psi, [2, 3])],
                          [qtl.stokes_parameter(psi, [3, 1]), 
                           qtl.stokes_parameter(psi, [3, 2]),
                           qtl.stokes_parameter(psi, [3, 3])]])
print(stokes_matrix)

[[-0.6  0.   0. ]
 [ 0.   0.8  0. ]
 [ 0.   0.   0.8]]


We will now boost the two-fermion state, first in a manner that induces a helicity flip. For this, we retrieve the 3-velocity of the first Dirac fermion:

In [7]:
beta_1 = qtl.ThreeVector.beta(pmu_1)

By boosting the single particle states with a boost vector $\alpha\boldsymbol{\beta}_1$ with $\alpha\in\mathbb{R}$ and $\alpha > 0$, we can investigate the behavior of the helicity eigenstates for $\alpha \approx 1$. If $\alpha>1$, we should observe a helicity flip, and if $\alpha<1$ this should not be observed.

In [8]:
# Factor by which the boost vector is multiplied
alpha = 1.1

# Boost the single-particle states
l_p1b = qtl.boost(l_p1, alpha * beta_1)
l_p2b = qtl.boost(l_p2, alpha * beta_1)
r_p1b = qtl.boost(r_p1, alpha * beta_1)
r_p2b = qtl.boost(r_p2, alpha * beta_1)
v_p1b = qtl.boost(v_p1, alpha * beta_1)
v_p2b = qtl.boost(v_p2, alpha * beta_1)

If `alpha` is larger than unity, we expect that the handedness of the first fermion remains unchanged, while the handedness of the second fermion is flipped:

In [9]:
print("fermion 1:", np.round(l_p1b.ket))
print("fermion 2:", np.round(l_p2b.ket))

fermion 1: [1.-0.j 0.-0.j]
fermion 2: [ 0.+0.j -0.+1.j]


From the boosted single-particle states, we construct our boosted mixed state:

In [10]:
# Pure components
psi_1b = 1j * (l_p1b * l_p2b - r_p1b * r_p2b) / np.sqrt(2)
psi_2b = v_p1b * v_p2b
pures_b = [psi_1b, psi_2b]

# Mixed state
psi_b = qtl.QuantumState.mixed(pures_b, weights)

The boosted mixed state has a density matrix

In [11]:
print(np.round(psi_b.rho, 2))

[[ 0.05+0.j -0.05-0.j -0.05+0.j  0.05-0.j]
 [-0.05+0.j  0.45+0.j -0.35-0.j -0.05-0.j]
 [-0.05-0.j -0.35+0.j  0.45-0.j -0.05+0.j]
 [ 0.05+0.j -0.05+0.j -0.05-0.j  0.05-0.j]]


which, if $\alpha > 1$, differs from the unboosted density matrix. If we boost a helicity-momentum eigenstate in some arbitrary direction, i.e. not in the direction of its momentum, we obtain a mixture of helicity-momentum eigenstates:

In [12]:
# Arbitrary boost vector
beta = qtl.ThreeVector(0.6, 1.1, 2.2)

# Boosted left-handed massive Dirac fermion
l_p1_beta = qtl.boost(l_p1, beta)
print(l_p1_beta.ket)

[ 0.64314432-0.33921143j -0.04476236-0.68505279j]


## Photons

Here we will consider quantum states of photons. We first define light-like 4-momenta $k_\pm = |\mathbf{k}|(1,\pm\hat{\mathbf{k}})$ and corresponding single-particle momentum eigenstates:

In [13]:
# Light-like 4-momentum kmu
k = 2
kmu_1 = qtl.FourVector(k, k, 0.2, 0.4)
kmu_2 = -kmu_1

# Massless single-particle states
l_k1 = qtl.QuantumState.single(kmu_1, "L")
l_k2 = qtl.QuantumState.single(kmu_2, "L")
r_k1 = qtl.QuantumState.single(kmu_1, "R")
r_k2 = qtl.QuantumState.single(kmu_2, "R")
h_k1 = qtl.QuantumState.single(kmu_1, "H")
h_k2 = qtl.QuantumState.single(kmu_2, "H")
v_k1 = qtl.QuantumState.single(kmu_1, "V")
v_k2 = qtl.QuantumState.single(kmu_2, "V")

The helicity of a photon is Lorentz invariant. When boosted, a photon's helicity eigenstate vector solely acquires a helicity-dependent phase $e^{i\lambda\theta}$ where $\lambda$ is the helicity and $\theta$ is the phase (see _Quantum theory of fields, volume I: Foundations_ by S. Weinberg).

In [14]:
l_k1_beta = qtl.boost(l_k1, beta)
print(l_k1_beta.ket)

[0.08625176-0.99627337j 0.        +0.j        ]


Consider e.g.

$$|\Psi\rangle = \frac{1}{\sqrt{2}}\Big( |k_+,\mathrm{H};k_-,\mathrm{V}\rangle - |k_+,\mathrm{V};k_-,\mathrm{H}\rangle \Big)\,,$$

which we create and boost with the following lines:

In [15]:
# Two-photon Bell state
photons = (h_k1 * v_k2 - v_k1 * h_k2) / np.sqrt(2)

# Boosted horizontal and verticle single-particle modes
h_k1b = qtl.boost(h_k1, beta)
h_k2b = qtl.boost(h_k2, beta)
v_k1b = qtl.boost(v_k1, beta)
v_k2b = qtl.boost(v_k2, beta)

# Boosted two-photon Bell state
photons_b = (h_k1b * v_k2b - v_k1b * h_k2b) / np.sqrt(2)

Here we will compare the Stokes matrices of $|\Psi\rangle$ and $U(\Lambda)|\Psi\rangle$:

In [16]:
photons_stokes = np.array([[qtl.stokes_parameter(photons, [1, 1]), 
                            qtl.stokes_parameter(photons, [1, 2]),
                            qtl.stokes_parameter(photons, [1, 3])],
                           [qtl.stokes_parameter(photons, [2, 1]), 
                            qtl.stokes_parameter(photons, [2, 2]),
                            qtl.stokes_parameter(photons, [2, 3])],
                           [qtl.stokes_parameter(photons, [3, 1]), 
                            qtl.stokes_parameter(photons, [3, 2]),
                            qtl.stokes_parameter(photons, [3, 3])]])

photons_stokes_b = np.array([[qtl.stokes_parameter(photons_b, [1, 1]), 
                              qtl.stokes_parameter(photons_b, [1, 2]),
                              qtl.stokes_parameter(photons_b, [1, 3])],
                             [qtl.stokes_parameter(photons_b, [2, 1]), 
                              qtl.stokes_parameter(photons_b, [2, 2]),
                              qtl.stokes_parameter(photons_b, [2, 3])],
                             [qtl.stokes_parameter(photons_b, [3, 1]), 
                              qtl.stokes_parameter(photons_b, [3, 2]),
                              qtl.stokes_parameter(photons_b, [3, 3])]])

print("Original:")
print(photons_stokes)
print()
print("Boosted:")
print(photons_stokes_b)

Original:
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]

Boosted:
[[-0.52068348  0.85374979  0.        ]
 [-0.85374979 -0.52068348  0.        ]
 [ 0.          0.         -1.        ]]


We see that the linear polarization modes mix due to the helicity dependence on the acquired phase, while the circular polarization modes (definite helicity) are conserved.